# 📌 Google Cloud Natural Language API 기반 감정 분석 자동화

## 🚀 기능
- **43만 개 리뷰 데이터를 Google Cloud Natural Language API로 감정 분석 라벨링**
- **멀티스레딩 + 배치(1000개 단위) 처리로 속도 최적화**
- **1000개 처리 후 자동 CSV 저장 (`reviews_labeled_processed.csv`), 중간 작업 중지로 인한 리스크 방지**
- **API 요청 속도 조절 및 Rate Limit(429 오류) 방지**
- **API 요청 초과 시 자동 재시도**

---

## ⏳ **예상 처리 시간**
📌 **전체 예상 소요 시간:** **약 16시간 35분**  

---


In [ ]:
#테스트

import os
from google.cloud import language_v1


os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "C:/Users/user/Downloads/project-test-452001-7fe438a2af3d.json"# JSON 키 파일 경로 직접 지정
client = language_v1.LanguageServiceClient() # 클라이언트 생성

print("Google Cloud Natural Language API 연결 성공! 🚀")

Google Cloud Natural Language API 연결 성공! 🚀


In [ ]:
from google.cloud import language_v1

# 테스트 함수
def analyze_sentiment(text):
    client = language_v1.LanguageServiceClient()

    document = language_v1.Document(
        content=text,
        type_=language_v1.Document.Type.PLAIN_TEXT,
        language="ko"
    )

    # 감성 분석 요청
    response = client.analyze_sentiment(request={'document': document})
    sentiment = response.document_sentiment

    print(f"감정 점수: {round(sentiment.score,2)}, 감정 강도: {round(sentiment.magnitude,2)}")     # 결과 출력

    # 감정 상태 해석
    if sentiment.score > 0.3:
        print("😃 긍정")
    elif sentiment.score < -0.3:
        print("😡 부정")
    else:
        print("😐 중립")

In [ ]:
# 테스트할 한글 문장
text_content = "배송 30일 걸려요"

# 감성 분석 실행
analyze_sentiment(text_content)

감정 점수: -0.2, 감정 강도: 0.2
😐 중립


### csv에서 데이터 가져와서 라벨링 작업 진행, 라벨링 속도를 높이기 위해 멀티 코어 사용

In [ ]:
import pandas as pd
import os
import threading
from google.cloud import language_v1
from concurrent.futures import ThreadPoolExecutor, as_completed

# Google Cloud 인증 설정
GOOGLE_CREDENTIAL_PATH = r"C:\Users\user\Downloads\project-test-452001-7fe438a2af3d.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GOOGLE_CREDENTIAL_PATH

# 진행 상황을 추적하는 전역 변수
progress_count = 0
lock = threading.Lock()  # 멀티스레딩 동기화용 Lock

# 저장할 CSV 파일명
output_csv = r"G:\내 드라이브\자료\멀티캠퍼스\최종 프로젝트\reviews_labeled_processed.csv"

def analyze_sentiment(text):
    """ 개별적으로 감정 분석을 수행하는 함수 (멀티스레딩용) """
    if pd.notna(text):  # NaN 값 방지
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GOOGLE_CREDENTIAL_PATH  # 인증 재설정
        client = language_v1.LanguageServiceClient()  # 각 스레드에서 클라이언트 생성
        document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
        sentiment = client.analyze_sentiment(request={"document": document}).document_sentiment
        return round(sentiment.score, 2), round(sentiment.magnitude, 2)
    return None, None

def process_row(index, text):
    """ 개별적으로 감정 분석을 수행하고, (index, 결과) 반환 """
    global progress_count
    score, magnitude = analyze_sentiment(text)

    # 진행 상황 업데이트 (동기화)
    with lock:
        progress_count += 1
        print(f"\r진행 상황: {progress_count}/{total_rows}", end="")

    return index, score, magnitude

if __name__ == "__main__":
    # 데이터 불러오기 (전체 데이터)
    df = pd.read_csv(r"G:\내 드라이브\자료\멀티캠퍼스\최종 프로젝트\reviews_labeled.csv")

    # 감정 분석 결과 컬럼 추가
    df["label_score"] = None
    df["label_magnitude"] = None

    # 총 데이터 개수
    total_rows = len(df)

    # 병렬 처리 설정 (CPU 7개 사용)
    max_workers = 7

    # 저장 파일 존재 여부 확인
    file_exists = os.path.exists(output_csv)

    # 1000개 단위로 처리
    for start_idx in range(0, total_rows, 1000):
        end_idx = min(start_idx + 1000, total_rows)
        df_batch = df.iloc[start_idx:end_idx].copy()

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_index = {executor.submit(process_row, i, text): i for i, text in df_batch["sentence"].items()}

            for future in as_completed(future_to_index):
                i, score, magnitude = future.result()
                df_batch.at[i, "label_score"] = score
                df_batch.at[i, "label_magnitude"] = magnitude

        # CSV에 저장 (1000개씩 추가 저장)
        df_batch.to_csv(output_csv, mode='a', index=False, header=not file_exists)
        file_exists = True  # 첫 번째 저장 후, 이후에는 헤더 추가 안 함

        print(f"\n🔹 {end_idx}/{total_rows}개 저장 완료!")

    print("\n✅ 감정 분석 및 저장 완료!")

In [ ]:
model_path = "G://내 드라이브//자료//멀티캠퍼스//최종 프로젝트//best_model_with_val(250227).pth"
model = AutoModelForSequenceClassification.from_pretrained("monologg/kobert", num_labels=3)
model.load_state_dict(torch.load(model_path, map_location="cuda", weights_only=True), strict=False)

In [ ]:
# ✅ 최적화 설정 (학습률 1.5e-5, L2 정규화 weight_decay 증가)
optimizer = AdamW(model.parameters(), lr=1.56e-5, weight_decay=0.025)  # 🔥 weight_decay 증가

# ✅ 학습 스케줄러 설정 (Cosine Decay 개선)
lr_scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=100,
    num_training_steps=len(train_loader) * 12,  # 12 Epoch
)

# ✅ Gradient Accumulation 적용 (메모리 최적화)
gradient_accumulation_steps = 4  # 🔥 추가
num_epochs = 12  # 🔥 Epoch 유지
best_loss = float("inf")  # ✅ 최적 모델 저장용